# Task 1: Data Collection & Preprocessing

## Step 1: Setup and Imports

In [ ]:
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Add scripts to path
sys.path.append('../scripts')

from scraper import EthiopianBankScraper
from preprocessing import ReviewPreprocessor

print(" Libraries imported successfully!")

## Step 2: Verify Bank Apps

In [ ]:
# Test if we can access the banking apps
scraper = EthiopianBankScraper()
scraper.verify_apps()

## Step 3: Scrape Reviews

In [ ]:
print(" Starting review scraping...")
raw_df = scraper.scrape_all_banks()

print("\n Sample of raw data:")
display(raw_df.head(3))

## Step 4: Preprocess Data

In [ ]:
print(" Starting data preprocessing...")
preprocessor = ReviewPreprocessor()
success = preprocessor.process()

if success:
    clean_df = preprocessor.df
    print(" Preprocessing completed!")
    display(clean_df.head(3))

## Step 5: Visualize Results

In [ ]:
# Set up plotting
plt.style.use('seaborn')
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 1. Reviews per bank
bank_counts = clean_df['bank_name'].value_counts()
axes[0,0].bar(bank_counts.index, bank_counts.values, color=['#1f77b4', '#ff7f0e', '#2ca02c'])
axes[0,0].set_title('Reviews per Bank')
axes[0,0].tick_params(axis='x', rotation=45)

# 2. Rating distribution
rating_counts = clean_df['rating'].value_counts().sort_index()
axes[0,1].pie(rating_counts.values, labels=rating_counts.index, autopct='%1.1f%%')
axes[0,1].set_title('Rating Distribution')

# 3. Reviews over time
clean_df['review_date'] = pd.to_datetime(clean_df['review_date'])
monthly = clean_df.groupby([pd.Grouper(key='review_date', freq='M'), 'bank_name']).size().unstack()
monthly.plot(ax=axes[1,0])
axes[1,0].set_title('Reviews Over Time')
axes[1,0].tick_params(axis='x', rotation=45)

# 4. Text length distribution
sns.histplot(data=clean_df, x='text_length', ax=axes[1,1])
axes[1,1].set_title('Review Text Length Distribution')

plt.tight_layout()
plt.show()

## Step 6: Final Verification

In [ ]:
print(" The outcome of TASK 1!")
print(f" Total reviews: {len(clean_df)}")
print(f" Banks covered: {clean_df['bank_name'].unique().tolist()}")
print(f" Date range: {clean_df['review_date'].min()} to {clean_df['review_date'].max()}")
print(f"⭐ Ratings: {clean_df['rating'].min()} to {clean_df['rating'].max()}")

# Check if we met requirements
bank_counts = clean_df['bank_name'].value_counts()
target_met = all(count >= 400 for count in bank_counts)
print(f" 400+ reviews per bank: {' ACHIEVED' if target_met else ' NOT MET'}")